## cargamos el pandas

In [1]:
import pandas as pd
import numpy as np

para poder pintar con plot pylab

In [3]:
%pylab inline #esto permite a pandas pintar en el notebook

UsageError: unrecognized arguments: #esto permite a pandas pintar en el notebook


In [4]:
train = pd.read_csv('C:/Users/Juanpe/MachineLearningTest/data/train.csv')

In [ ]:
train.head(22)

In [ ]:
train.age.plot()

In [ ]:
train['age'].hist()

In [ ]:
train.hist(figsize=(20,20))

In [ ]:
train.boxplot('age')

In [ ]:
train.describe()

In [ ]:
train.iloc[21,2]

In [ ]:
train.loc[21,'age']

In [ ]:
train.shape

In [ ]:
train.info()

In [43]:
nrow = train.shape[0]

In [44]:
len(train)

264

In [ ]:
train.num.unique()

vamos a hacer un for

In [6]:
for i in range(nrow):
    if train['num'][i] != 0:
        train.loc[i,'num'] = 1
        

In [ ]:
train

In [42]:
range(nrow)

range(0, 264)

In [ ]:
train['num'][train['num']!=0]

In [ ]:
train.loc[2,'num']

In [ ]:
train

In [ ]:
train[train['num']!=0]

## Preprocessing 

http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values

#nos saltamos la estandarización
#vemos valores Nan

In [ ]:
pd.isnull(train).any(1).nonzero()[0]

In [ ]:
train.iloc[pd.isnull(train).any(1).nonzero()[0]]

In [ ]:
train.loc[[87, 166, 192],:]

#pillamos la librería q imputa valores

In [7]:
train = train.fillna(train.mean())#forma simplificada de Juan

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train)

nos crea una matriz sin nombres de campo, por eso lo de columns

In [ ]:
train = pd.DataFrame(imp.transform(train),columns=train.columns)

In [ ]:
train

## vamos a ver cada variable % sanos

In [ ]:
train.groupby('num').size()

In [ ]:
train.num.mean() #hay un 43% de enfermos

In [ ]:
train.sex.mean() #hay un 67% de hombres

In [ ]:
train.groupby('sex').agg({'num': np.mean}) #sexo importa

In [ ]:
train.groupby('cp').agg({'num': np.mean}) # cp importa mucho

In [ ]:
train.groupby('fbs').agg({'num': np.mean}) #nada castaña pilonga

In [ ]:
train.groupby('restecg').agg({'num': np.mean}) # importa poco

In [ ]:
train.groupby('exang').agg({'num': np.mean}) # importa bastante

In [ ]:
train.groupby('slope').agg({'num': np.mean}) # importa bastante

In [ ]:
train.groupby('ca').agg({'num': np.mean}) #importa demasiado!!

In [ ]:
train.groupby('thal').agg({'num': np.mean}) # importa bastante

## voy a pintar un mapa de correlaciones de variables

In [ ]:
#correlation map
import seaborn as sns
import matplotlib.pyplot as plt
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

sale lo q habia visto yo con los group by!!!!!!

## modelo encontrado por Miguel para ver importancia de variables

http://python-apuntes.blogspot.com.es/2016/07/importancia-variables.html

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
# CONVIERTE DATA.FRAME EN MATRIZ npy
# -------------------------------------------------------------------
dataset = train
clase_name = 'num' # nombre de variable a predecir
headers = dataset.columns.values.tolist()
headers.remove(clase_name)
clase   = dataset[clase_name].tolist() # array con variable a predecir
clase   = np.array(clase)


# MODELO
# -------------------------------------------------------------------
model = ExtraTreesClassifier()
model.fit(dataset, clase)


In [ ]:
# IMPORTANCIA VARIABLES
# -------------------------------------------------------------------
var_imp = pd.DataFrame[{
 'feature':headers, 
 'v_importance':model.feature_importances_.tolist()
 }]
var_imp.sort_values(by = 'v_importance', ascending=False)

## lanzamos modelo naive bayes

separamos los datos X, y (lo pasamos a forma matriz por Victor manía)

In [8]:
X, y = train.iloc[:,:-1].as_matrix(), train.iloc[:,-1].as_matrix()

y tb en train y test de ambas variables

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train


## lanzamos modelo. copio y pego del scikit learn la forma

In [11]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test) 

#print("Number of mislabeled points out of a total %d points : %d"% (X_train.shape[0],(y_train != y_pred).sum())) Number of mislabeled points out of a total 150 points : 6

## vemos la matriz de confusión

In [12]:
from sklearn.metrics import confusion_matrix

In [28]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

In [15]:
cnf_matrix

array([[39,  5],
       [ 6, 38]], dtype=int64)

In [29]:
pd.DataFrame(cnf_matrix,columns=['no','si'], index=['no','si'])

,no,si
no,39,5
si,6,38


In [19]:
gnb.get_params()

{'priors': None}

## vamos a mejorar el modelo. hay muchas formas. vamos a hacer 2 de ellas. 1) PCA - analisis de componentes principales 2) Iterative train-test scikit learn

## 1) PCA

In [37]:
from sklearn.decomposition import PCA

pca = PCA(9) # lo vamos a poner en 9 porq a Pablo le ha salido bien así, pero lo suyo es hacerlo iterativo

Xtr =pca.fit(X).transform(X)

Xtr_train, Xtr_test, ytr_train, ytr_test = train_test_split(Xtr, y, test_size=0.33)

In [40]:
#lanzamos modelo
gnbtr = GaussianNB()

ytr_pred = gnb.fit(Xtr_train, ytr_train).predict(Xtr_test) 

In [41]:
# Compute confusion matrix
cnftr_matrix = confusion_matrix(ytr_test, ytr_pred)
confussiontr = pd.DataFrame(cnftr_matrix,columns=['no','si'], index=['no','si'])
confussiontr


,no,si
no,43,6
si,9,30


## 2) train-test iterativo

lo q queremos es comparar la predicción del modelo con diferentes formas de dividir en train y test

In [50]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

def traintest (porcentaje, iteraciones):
    results = []
    for i in range(iteraciones):
        # dividimos al porcentaje calculado% 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= porcentaje, random_state=42)

        # lanzamos modelo
        gnb = GaussianNB()
        y_pred = gnb.fit(X_train, y_train).predict(X_test)

        # Compute confusion matrix
        cnf_matrix = confusion_matrix(y_test, y_pred)

        # guardamos matriz
        matriz = pd.DataFrame(cnf_matrix,columns=['no','si'], index=['no','si'])

        results.append([matriz, y_pred, y_test, X_test, y_train, X_train])
    return results

In [51]:
RES = traintest(0.33,10)

In [52]:
for i in range(len(RES)):
    print(RES[i][0])

    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38
    no  si
no  39   5
si   6  38


In [46]:
range(10)

range(0, 10)

## qué peso da el modelo a cada variable?  
http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [20]:
gnb.score(X_test, y_test)

0.875

In [21]:
gnb.score(X_train, y_train)

0.86363636363636365